    ==============================================================================================

## Importing the DataFrames
- ``` df = pd.read_pickle("file_name.pkl")```  --> create 1 df from a pkl file

In [6]:
import pandas as pd
import numpy as np
pd.set_option("display.min_rows", 0) 
pd.set_option("display.max_rows", 30) 
pd.__version__

'1.4.4'

In [11]:

ol = pd.read_pickle("clean_Tables\OrderLines_c.pkl")
ord = pd.read_pickle("clean_Tables\Orders_c.pkl")

pro = pd.read_pickle("clean_Tables\Products_c.pkl")

In [12]:
ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ORD_ID         226909 non-null  int64  
 1   ordreatDate    226909 non-null  object 
 2   ord_TotalPaid  226904 non-null  float64
 3   ord_State      226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [13]:
ol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   OL_ID             293983 non-null  int64 
 1   ol_ORD_ID         293983 non-null  int64 
 2   ol_ProdQuantity   293983 non-null  int64 
 3   ol_sku            293983 non-null  object
 4   ol_ProdUnitPrice  293983 non-null  object
 5   ol_ProcessDate    293983 non-null  object
dtypes: int64(3), object(3)
memory usage: 13.5+ MB


In [14]:
pro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10579 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PRO_SKU         10579 non-null  object
 1   pro_Name        10579 non-null  object
 2   pro_Desc        10572 non-null  object
 3   pro_Price       10534 non-null  object
 4   pro_PromoPrice  10579 non-null  object
 5   pro_InStock     10579 non-null  int64 
 6   pro_Type        10529 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


    ==============================================================================================

## Merge DataFrames
- ``` df.shape``` , ``` df.size``` , ``` df.ndim``` 
- ``` df.sample(5)``` , ``` df.info()``` 
- Numerical : ``` df.describe()``` , ``` df.col.nlargest()``` , ``` df.col.nsmallest()``` 
- Category : ``` df.nunique()``` , ``` df.unique() ``` 

    ==============================================================================================

## Clean Merged Data
- Remember to create a copy of the df using ``` df.copy()``` 

In [ ]:
pro_c = pro.copy()

      ===============================================

### Drop duplicated Columns , Set Index
 - ``` df.columns```   , ``` df.index``` 
 - ``` df=df.rename(columns={"A": "a", "B": "c"})``` 
 - ``` df.columns = ["a","b":"x"]``` 
     - take care, renaming the columns like that will convert the NAN to some value!!
 - ``` df=df.set_index("col")```  , ``` df=df.reset_index()``` 

In [ ]:
pro_c.columns

In [ ]:
pro_c=pro_c.rename(columns={"sku": "pro_SKU"
                        , "name": "pro_Name"
                        , "desc": "pro_Desc"
                        , "price": "pro_Price"
                        , "promo_price": "pro_PromoPrice"
                        , "in_stock": "pro_InStock"
                        , "type": "pro_Type"})

In [ ]:
pro_c.columns

In [ ]:
pro_c.index #hint: no need to change index

      ===============================================
      
### Clean NAN and empty cells
- ``` df.isna().sum()``` 
- check if it makse sense to replace empty cells with NAN or with another value!
- ``` df = df.replace('^\s*$', np.nan)```  -->replace empty cells and cells with only whitspace with NAN
- ``` df=df.col.fillna(value,method="bfill"or"ffill",limit=value)``` 

- Extra: 
  - ``` (df.values == '').sum()```  --> check if any cell is empty
  - ``` df.col.str.isspace().sum()```  --> check if all cell is filled with whitespaces

___ For Order.csv___

In [ ]:
ord_c.sample(5)

In [ ]:
ord_c.isna().sum()

In [ ]:
ord_c.info()

In [ ]:
import numpy as np
ord_c = ord_c.replace('^\s*$', np.nan) #Question is it safe to do that?


In [ ]:
ord_c.isna().sum()

In [ ]:
ord_c.ord_total_paid.dtype

In [ ]:
ord_c.ord_total_paid=ord_c.ord_total_paid.fillna(0.0)

In [ ]:
ord_c.ord_total_paid.dtype

In [ ]:
ord_c.isna().sum()

___ For OrderLines.csv ____

In [ ]:
ol_c.isna().sum()

In [ ]:
ol_c.info()

In [ ]:

ol_c = ol_c.replace('^\s*$', np.nan)


In [ ]:
ol_c.isna().sum()

___ For Products.csv ____

In [ ]:
pro_c.isna().sum()

In [ ]:
pro_c.info()

In [ ]:
(pro_c.values == '').sum()

In [ ]:

pro_c = pro_c.replace('^\s*$', np.nan)


In [ ]:
pro_c.isna().sum()

Fix NAN from pro_Desc        

In [ ]:
pro_c.pro_Desc=pro_c.pro_Desc.fillna("No Description")

Fix NAN from pro_Type           

In [ ]:
pro_c.loc[pro_c.pro_Type == "0"]
# No value is Zero , safelly set NAN to 0

In [ ]:
pro_c.pro_Type = pro_c.pro_Type.fillna("0")

In [ ]:
pro_c.isna().sum()

Fix NAN from pro_Price   

In [ ]:
pro_c.loc[pro_c.pro_Price == "0.0"]
# No value is Zero , safelly set NAN to 0

In [ ]:
pro_c.pro_Price = pro_c.pro_Price.fillna("0.0")

In [ ]:
pro_c.isna().sum()

      ===============================================
### Fix DataTypes
- use ```df.nunique()``` to find if column is bool or Category
- before change, make sure all values will be converted correct
  - to convert to numeric : check no letters, 
      - if float(2dec) ```(~df.col.str.contains("^\d+.\d{2}$")).sum()```
      - if integer ```(~df.col.str.contains("^\d+$")).sum()```
  - to convert to category : check there are reasonable number of unique items
  - to convert to Datetime : ```(~df.col.str.contains("^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$")).sum()```
- ``` df.col.astype(type,errors="raise")``` 
  - type = "int","float","bool","category","object","datetime","timedelta"
- for mixed data
  - ``` pd.to_numeric(df.col, downcast=x,errors="raise") ``` 
  x = "integer" or "float"
  - ``` pd.to_datetime(df.col, downcast=None,errors="raise") ``` 
  - ``` pd.to_timedelta(df.col, downcast=None,errors="raise") ``` 

___ For Orders.csv ____

In [ ]:
ord_c.info()

In [ ]:
ord_c.ord_created_date=pd.to_datetime(ord_c.ord_created_date,errors="raise")

In [ ]:
ord_c.info()

In [ ]:
ord_c.ord_state=ord_c.ord_state.astype("category",errors="raise")

In [ ]:
ord_c.info()

___ For OrderLines.csv ____

In [ ]:
ol_c.info()
#ol_ProcessDate need to be changed to

____  Convert ol_ProcessDate to datetime  _______

In [ ]:
(~ol_c.ol_ProcessDate.str.contains("^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$")).sum() 
# all rows has the datetime pattern, save to convert

In [ ]:
ol_c.ol_ProcessDate = pd.to_datetime(ol_c.ol_ProcessDate, errors="raise")

In [ ]:
ol_c.ol_ProcessDate.sample(10)

____  Convert ol_ProdUnitPrice to float  _______

In [ ]:
ol_c.info()

In [ ]:
ol_c.ol_ProdUnitPrice.sample(10)

Check if some values are not of float pattern

In [ ]:
(~ol_c.ol_ProdUnitPrice.str.contains("^\d+.\d{2}$")).sum()
#36170 rows doesn't has the correct pattern

In [ ]:
ol_c.ol_ProdUnitPrice[(~ol_c.ol_ProdUnitPrice.str.contains("^\d+.\d{2}$"))].sort_values()
# can not convert as:
# there is 1 negative value, make it +ve
# for the values 6.543.59 convert it to 6543.59

Get rid of the negative value

In [ ]:
ol_c.ol_ProdUnitPrice = ol_c.ol_ProdUnitPrice.str.replace("-","")


In [ ]:
ol_c.ol_ProdUnitPrice.str.contains("-").sum()

Get rid of the first dot 3.112.23 --> 3112.23

In [ ]:
ol_c.info()

In [ ]:
ol_c.loc[ol_c.ol_ProdUnitPrice.str.contains("\d+\.\d{3}\.\d{2}")]

In [ ]:
ol_c.ol_ProdUnitPrice.str.replace(r"(\d+)\.(\d{3})(\.\d{2})",lambda m: m.group(1) + m.group(2) + m.group(3))


In [ ]:
ol_c=(
    ol_c
    .assign(ol_ProdUnitPrice = lambda x:np.where(
      x.ol_ProdUnitPrice.str.contains('\d+\.\d{3}\.\d{2}'),
      x.ol_ProdUnitPrice.str.replace(r"(\d+)\.(\d{3})(\.\d{2})"
                ,lambda m: m.group(1) + m.group(2) + m.group(3)), 
      x.ol_ProdUnitPrice
      ))
)

In [ ]:

# (
# ol_c
#   .assign(
#     problem_number = lambda x: np.where(
#       # condition to find out the error
#       x['unit_price'].str.contains('\d.'),
#       # apply the solution when you find the error 
#       x['unit_price'].str.replace('-','',1), 
#       # leave the unit price as it was, as it doesn't have any error
#       x['unit_price']
#       )
#   )
#   #.query("id_order == 365886")
# )

Converting the col to float

In [ ]:
ol_c.ol_ProdUnitPrice = pd.to_numeric(ol_c.ol_ProdUnitPrice, errors="raise")

In [ ]:
ol_c.info()

___ For Products.csv ____

In [ ]:
pro_c.info()
#ol_ProcessDate need to be changed to

____  Convert pro_Price  to float  _______

In [ ]:
pro_c.info()

In [ ]:
pro_c.pro_Price.sample(10)

Check if some values are not of float pattern

In [ ]:
(~pro_c.pro_Price.str.contains("^\d+.\d{2}$")).sum()
#3134 rows doesn't has the correct pattern

In [ ]:
pro_c.pro_Price[(~pro_c.pro_Price.str.contains("^\d+.\d{2}$"))].sort_values()
# for the values 6.543.59 convert it to 6543.59

Get rid of the first dot 3.112.23 --> 3112.23

In [ ]:
pro_c.loc[pro_c.pro_Price.str.contains("\d+\.\d{3}\.\d{2}")]

In [ ]:
#pro_c.ol_ProdUnitPrice.str.replace(r"(?P<one>\d+).(?P<two>\d{3})(?P<three>.\d{2})"
#                                  ,lambda m: m.group("one") + m.group("two") + m.group("three"))


In [ ]:
pro_c=(
    pro_c
    .assign(ol_ProdUnitPrice = lambda x:np.where(
      x.ol_ProdUnitPrice.str.contains('\d+\.\d{3}\.\d{2}'),
      x.ol_ProdUnitPrice.str.replace(r"(\d+)\.(\d{3})(\.\d{2})"
                ,lambda m: m.group(1) + m.group(2) + m.group(3)), 
      x.ol_ProdUnitPrice
      ))
)

In [ ]:

# (
# pro_c
#   .assign(
#     problem_number = lambda x: np.where(
#       # condition to find out the error
#       x['unit_price'].str.contains('\d.'),
#       # apply the solution when you find the error 
#       x['unit_price'].str.replace('-','',1), 
#       # leave the unit price as it was, as it doesn't have any error
#       x['unit_price']
#       )
#   )
#   #.query("id_order == 365886")
# )

Converting the col to float

In [ ]:
pro_c.ol_ProdUnitPrice = pd.to_numeric(pro_c.ol_ProdUnitPrice, errors="raise")

In [ ]:
pro_c.info()

    ==============================================================================================

## Re-Explore the data
draw some ``` df.col.hist()```  , ``` df.ser.boxplot()```  per column
take notes

In [ ]:
pro_c.sample(10)  # Question: how state="Shopping Basket" while "Total_paid" has a value!! 

    ==============================================================================================

## Answer Bussines questions
